In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Read the CSV file and skip the first row
df2 = pd.read_csv('/content/drive/MyDrive/Tower_Sentiment.csv', skiprows=0)

# Drop columns that do not need to be read
df2.drop(df2.columns[[4, 5]], axis=1, inplace=True)
df2.head()

<ipython-input-3-2ceac9fa97fd>:4: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/drive/MyDrive/Tower_Sentiment.csv', skiprows=0)


,Conversation ID,Source,Text,Timestamp,ID,Sentiment,new,FullSentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,user,I,2023-04-17T21:15:34.492,00036309-c615-45b0-bd95-d4669b72e66b,Neutral,00036309-c615-45b0-bd95-d4669b72e66b,Neutral
1,00036309-c615-45b0-bd95-d4669b72e66b,bot,Hmm.. sorry I couldn't find an answer for that!,2023-04-17T21:16:08.778,00036309-c615-45b0-bd95-d4669b72e66b,NaN,NaN,NaN
2,00036309-c615-45b0-bd95-d4669b72e66b,bot,"To have a guided conversation instead, please ...",2023-04-17T21:16:09.801,00036309-c615-45b0-bd95-d4669b72e66b,NaN,NaN,NaN
3,00036309-c615-45b0-bd95-d4669b72e66b,bot,Are you an existing Tower customer?,2023-04-17T21:16:10.82,00036309-c615-45b0-bd95-d4669b72e66b,NaN,NaN,NaN
4,00036309-c615-45b0-bd95-d4669b72e66b,user,Yes,2023-04-17T21:16:28.619,00036309-c615-45b0-bd95-d4669b72e66b,Neutral,NaN,NaN


In [ ]:
import pandas as pd

# Assuming your dataframe 'df2' and grouping logic as provided

# Group by 'Conversation ID' and aggregate 'Text' and 'FullSentiment'
def concatenate_text_and_sentiment(group):
    combined_text = ', '.join([f"{source.capitalize()}: {text}" for _, source, text in group[['Timestamp', 'Source', 'Text']].itertuples(index=False)])
    full_sentiment = group['FullSentiment'].iloc[0]  # Get FullSentiment from the first row of the group
    return pd.Series({
        'Conversation ID': group['Conversation ID'].iloc[0],
        'Combined_Text': combined_text,
        'FullSentiment': full_sentiment
    })

grouped_df = df2.groupby('Conversation ID').apply(concatenate_text_and_sentiment).reset_index(drop=True)

#FOR NOW
grouped_df.dropna(subset=['FullSentiment'], inplace=True)

print(grouped_df)

                          Conversation ID  \
0    00036309-c615-45b0-bd95-d4669b72e66b   
1    000715a4-1d49-4a54-8ee8-140808202926   
2    000c670b-4513-49d4-bf3a-daa371d4ef52   
3    000d5541-0607-463a-b05c-aeb888a69233   
4    0016ac21-a8ae-476d-aa75-8c4b62ce175a   
..                                    ...   
165  04007a95-5fbd-49b2-90f2-a11fa333759d   
166  0403ec9f-73a4-44b7-ad63-61d3cb7b671d   
167  040840df-c5da-4e12-9e52-56d3b7dd393b   
168  04168028-1a89-4cc4-b359-c3bac2f8b6fe   
169  04191067-272c-442d-8ac6-a7f29c538300   

                                         Combined_Text FullSentiment  
0    User: I, Bot: Hmm.. sorry I couldn't find an a...       Neutral  
1    User: I have a pending claim and the company f...       Neutral  
2    User: Hi there, Bot: Kia Ora! 👋, Bot: I can he...      Positive  
3    User: Hi I actually wanted to cancel my policy...      Positive  
4    User: Hi Charlie bot, im trying to add my airp...      Positive  
..                               

In [ ]:
import random
# Get the number of rows in the DataFrame
num_rows = len(grouped_df)

# Generate a random index within the valid range
random_index = random.randint(0, num_rows - 1)

# Get the text paragraph at the random index
text_paragraph = grouped_df['Combined_Text'][random_index]
sentiment = grouped_df['FullSentiment'][random_index]
conversation_ID = grouped_df['Conversation ID'][random_index]

print(conversation_ID)
print(text_paragraph)
print(sentiment)

02db8e05-a1ea-4c7a-80a8-cae7298c559f
User: do I need to provide car registration number for the car insurance, Bot: Changed the registration details for your car, boat, or trailer? We'll need to update your policy details 🧐, Bot: I can help you pass on the details to our team, they'll update your policy soon and send you a confirmation of the changes., Bot: Would you like to continue with this request?, User: I don't the car registration number, Bot: Changed the registration details for your car, boat, or trailer? We'll need to update your policy details 🧐, Bot: I can help you pass on the details to our team, they'll update your policy soon and send you a confirmation of the changes., Bot: Would you like to continue with this request?, User: no, Bot: Our team can help you with this! Let me find someone 👍, Bot: Sorry, our agents are offline right now 😔 Our agents are available Monday to Saturday from 9am, Bot: Contact us | After-hours Getting in touch late? Here's a few ways to reach ou

In [ ]:
import os
import openai

#do not use without consulting please
openai.api_key = ""
system_message = "You will be provided with text or conversation from a chat bot dataset, and your task is to analyze the sentiment of the users' interactions. Only output the sentiment and a small sentance as to why"


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system","content": system_message},
    {"role": "user","content": text_paragraph}
  ],
  temperature=0,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
output_content = response['choices'][0]['message']['content']

print("Text:")
print(text_paragraph)
print()

print("Sentiment:",output_content)
print("Annotated Sentiment: ", sentiment)

Text:
User: do I need to provide car registration number for the car insurance, Bot: Changed the registration details for your car, boat, or trailer? We'll need to update your policy details 🧐, Bot: I can help you pass on the details to our team, they'll update your policy soon and send you a confirmation of the changes., Bot: Would you like to continue with this request?, User: I don't the car registration number, Bot: Changed the registration details for your car, boat, or trailer? We'll need to update your policy details 🧐, Bot: I can help you pass on the details to our team, they'll update your policy soon and send you a confirmation of the changes., Bot: Would you like to continue with this request?, User: no, Bot: Our team can help you with this! Let me find someone 👍, Bot: Sorry, our agents are offline right now 😔 Our agents are available Monday to Saturday from 9am, Bot: Contact us | After-hours Getting in touch late? Here's a few ways to reach out to us.  | Send a message - Se

750 tokens = 0.10

750 = 1000 words

